In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm
from plotly.subplots import make_subplots
from distutils import util
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
# Jaime

df_calendar = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/calendar.csv")
df_listings = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/listings.csv")
df_neighbourhoods = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/neighbourhoods.csv")
df_reviews = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/reviews.csv")
df_reviews_det = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/reviews_detailed.csv")
df_listings_det = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/listings_detailed.csv")

# Preprocesado Calendar

In [4]:
df_calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,167183,2021-04-15,f,$45.00,$45.00,1.0,5.0
1,6369,2021-04-15,t,$60.00,$60.00,1.0,1125.0
2,6369,2021-04-16,t,$60.00,$60.00,1.0,1125.0
3,6369,2021-04-17,t,$60.00,$60.00,1.0,1125.0
4,6369,2021-04-18,t,$60.00,$60.00,1.0,1125.0


In [5]:
df_calendar.isna().sum()

listing_id           0
date                 0
available            0
price             1105
adjusted_price    1105
minimum_nights    1050
maximum_nights    1050
dtype: int64

In [6]:
df_calendar['listing_id'][df_calendar['price'].isna()].unique()

array([ 3197869, 23322783, 24438248, 33741664, 33949642, 36088162,
       40832825, 44046661, 45893742, 48223213], dtype=int64)

In [7]:
df_calendar['listing_id'][df_calendar['adjusted_price'].isna()].unique()

array([ 3197869, 23322783, 24438248, 33741664, 33949642, 36088162,
       40832825, 44046661, 45893742, 48223213], dtype=int64)

In [50]:
data = [
    go.Histogram(
        x = df_calendar['price'].unique(),
        opacity=0.6,
        name = "Precio"
    )
]

layout = go.Layout(title = "Distribución del precio", xaxis_title = "Precio", yaxis_title = "Frecuencia",
                   barmode = "overlay")

fig = go.Figure(data = data, layout = layout)

fig.show()

In [9]:
data = [
    go.Histogram(
        x = df_calendar['adjusted_price'].unique(),
        opacity=0.6,
        name = "Precio Ajustado"
    )
]

layout = go.Layout(title = "Distribución del precio ajustado", xaxis_title = "Precio Ajustado", yaxis_title = "Frecuencia",
                   barmode = "overlay")

fig = go.Figure(data = data, layout = layout)

fig.show()

Adjusted Price y Price son iguales --> Nos quedamos solo con Price

In [10]:
df_calendar_v2 = df_calendar.drop(columns="adjusted_price")

In [11]:
len(df_calendar_v2['listing_id'].unique())

19617

De un total de 19617 publicaciones, 10 tienen precio nulo (0,05%). Podemos por tanto eliminar estas publicaciones, ya que constituyen una mínima parte del dataset.

In [12]:
listings_nulos = []
for i in range(len(df_calendar_v2['listing_id'][df_calendar_v2['price'].isna()].unique())):
  listings_nulos.append(df_calendar_v2['listing_id'][df_calendar_v2['price'].isna()].unique()[i])

In [13]:
len(df_calendar_v2['listing_id'][df_calendar_v2['maximum_nights'].isna()].unique())

844

In [14]:
len(df_calendar_v2['listing_id'][df_calendar_v2['minimum_nights'].isna()].unique())

844

En este caso, el numero de listings es mayor, por lo que imputaremos la mediana como valor de referencia. 

In [15]:
df_calendar_v2['minimum_nights'] = df_calendar_v2['minimum_nights'].fillna(df_calendar_v2['minimum_nights'].median())
df_calendar_v2['maximum_nights'] = df_calendar_v2['maximum_nights'].fillna(df_calendar_v2['maximum_nights'].median())

In [16]:
df_calendar_v2.isna().sum()

listing_id           0
date                 0
available            0
price             1105
minimum_nights       0
maximum_nights       0
dtype: int64

Únicamente quedan los nulos del precio, los cuales vamos a eliminar.

In [17]:
df_calendar_v3 = df_calendar_v2.dropna()

In [18]:
df_calendar_v3.isna().sum()

listing_id        0
date              0
available         0
price             0
minimum_nights    0
maximum_nights    0
dtype: int64

In [19]:
df_calendar_v3.dtypes

listing_id          int64
date               object
available          object
price              object
minimum_nights    float64
maximum_nights    float64
dtype: object

Por ultimo, convertimos las variables a sus respectivos tipos, price a float, available a boolean y date a date

In [20]:
df_calendar_v3['date'] = pd.to_datetime(df_calendar_v3['date'])

C:\Users\jaime\AppData\Local\Temp/ipykernel_23040/2813003169.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
def precio_a_float(x):
  if(',' in x):
    x = x.replace(',', '')

  y = float(x.split('$')[1])
  return y

In [22]:
df_calendar_v3['price'] = df_calendar_v3['price'].apply(lambda x: precio_a_float(x))

C:\Users\jaime\AppData\Local\Temp/ipykernel_23040/2917744996.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
def available_to_bool(x):
  y = util.strtobool(x)
  return y

In [24]:
df_calendar_v3['available'] = df_calendar_v3['available'].apply(lambda x: available_to_bool(x))

C:\Users\jaime\AppData\Local\Temp/ipykernel_23040/1643907335.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
df_calendar_vf = df_calendar_v3.copy()

# Preprocesado Listings

In [26]:
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room , ensuite bathroom",13660,Simon,Chamartín,Hispanoamérica,40.45724,-3.67688,Private room,60,1,78,2020-09-20,0.58,1,180
1,21853,Bright and airy room,83531,Abdel,Latina,Cármenes,40.40381,-3.74130,Private room,31,4,33,2018-07-15,0.42,2,364
2,23001,Apartmento Arganzuela- Madrid Rio,82175,Jesus,Arganzuela,Legazpi,40.38840,-3.69511,Entire home/apt,50,15,0,NaN,NaN,7,1
3,24805,Gran Via Studio Madrid,346366726,A,Centro,Universidad,40.42183,-3.70529,Entire home/apt,92,5,10,2020-03-01,0.13,1,72
4,26825,Single Room whith private Bathroom,114340,Agustina,Arganzuela,Legazpi,40.38975,-3.69018,Private room,26,2,149,2020-03-12,1.12,1,365


In [27]:
df_listings.isna().sum()

id                                   0
name                                 3
host_id                              0
host_name                          527
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       5637
reviews_per_month                 5637
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [28]:
df_listings.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

Primero, convertimos last_review a formato date

In [29]:
df_listings_v2 = df_listings.copy()
df_listings_v2['last_review'] = pd.to_datetime(df_listings_v2['last_review'])

Por otro lado, la columna host_name, no influencia el precio por lo que no nos aporta información. En el caso de la columna name, al existir unicamente 3 nulos, borraremos dichas filas

In [30]:
df_listings_v3 = df_listings_v2.drop(columns="host_name")
df_listings_v3 = df_listings_v3[df_listings_v3['name'].notna()]

In [31]:
df_listings_v3.isna().sum()

id                                   0
name                                 0
host_id                              0
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       5634
reviews_per_month                 5634
calculated_host_listings_count       0
availability_365                     0
dtype: int64

Finalmente, para el caso de reviews per month, se trata de publicaciones sin reviews, por lo que las convertiremos a 0. Para el caso de last_review, tendremos que tomar una decisión

In [32]:
df_listings_vf = df_listings_v3.copy()
df_listings_vf['reviews_per_month'] = df_listings_vf['reviews_per_month'].fillna(0)

In [33]:
df_listings_vf.head()

,id,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room , ensuite bathroom",13660,Chamartín,Hispanoamérica,40.45724,-3.67688,Private room,60,1,78,2020-09-20,0.58,1,180
1,21853,Bright and airy room,83531,Latina,Cármenes,40.40381,-3.74130,Private room,31,4,33,2018-07-15,0.42,2,364
2,23001,Apartmento Arganzuela- Madrid Rio,82175,Arganzuela,Legazpi,40.38840,-3.69511,Entire home/apt,50,15,0,NaT,0.00,7,1
3,24805,Gran Via Studio Madrid,346366726,Centro,Universidad,40.42183,-3.70529,Entire home/apt,92,5,10,2020-03-01,0.13,1,72
4,26825,Single Room whith private Bathroom,114340,Arganzuela,Legazpi,40.38975,-3.69018,Private room,26,2,149,2020-03-12,1.12,1,365


# Preprocesado Detailed Reviews

In [3]:
df_reviews_det.isna().sum()

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      1
comments         352
dtype: int64

In [4]:
len(df_reviews_det['comments'])

625006

El número de reviews nulos es mínimo, por lo que borraremos dichas reviews


In [5]:
df_reviews_det_vf = df_reviews_det.dropna() 
df_reviews_det_vf.isna().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

In [6]:
df_reviews_det_vf['date'] = pd.to_datetime(df_reviews_det_vf['date'])
df_reviews_det_vf.dtypes

C:\Users\jaime\AppData\Local\Temp/ipykernel_20152/3860151056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_det_vf['date'] = pd.to_datetime(df_reviews_det_vf['date'])


listing_id                int64
id                        int64
date             datetime64[ns]
reviewer_id               int64
reviewer_name            object
comments                 object
dtype: object

Nos quedamos con las reviews que tengan una longitud menor a 512 caracteres, ya que el modelo NLP que emplearemos no acepta más de 512 caracteres.

In [7]:
df_reviews_det_vf = df_reviews_det_vf[df_reviews_det_vf['comments'].apply(lambda x: len(x)<512)]

Añadimos ahora un score por cada review. Para ello empleamos un modelo NLP de HuggingFace:

In [8]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading: 100%|██████████| 638M/638M [04:02<00:00, 2.76MB/s]


In [10]:
multilang_classifier = pipeline("sentiment-analysis", 
                                model=model, tokenizer = tokenizer)

In [11]:
df_reviews_det_vf['stars'] = df_reviews_det_vf['comments'].apply(lambda x: multilang_classifier(x)[0]['label'])

# Preprocesado Detailed Listings 

In [38]:
df_listings_det.isna().sum()

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
name                                               3
                                                ... 
calculated_host_listings_count                     0
calculated_host_listings_count_entire_homes        0
calculated_host_listings_count_private_rooms       0
calculated_host_listings_count_shared_rooms        0
reviews_per_month                               5637
Length: 74, dtype: int64

In [39]:
df_listings_det.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6369,https://www.airbnb.com/rooms/6369,20210413152844,2021-04-15,"Rooftop terrace room , ensuite bathroom",Excellent connection with the AIRPORT and EXHI...,NaN,https://a0.muscache.com/pictures/683224/4cc318...,13660,https://www.airbnb.com/users/show/13660,...,10.0,10.0,10.0,NaN,f,1,0,1,0,0.58
1,21853,https://www.airbnb.com/rooms/21853,20210413152844,2021-04-16,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,https://www.airbnb.com/users/show/83531,...,10.0,8.0,9.0,NaN,f,2,0,2,0,0.42
2,23001,https://www.airbnb.com/rooms/23001,20210413152844,2021-04-16,Apartmento Arganzuela- Madrid Rio,"Apartamento de tres dormitorios dobles, gran s...","Barrio Arganzuela, junto a Madrid Rio, zonas c...",https://a0.muscache.com/pictures/58e6a770-509c...,82175,https://www.airbnb.com/users/show/82175,...,NaN,NaN,NaN,NaN,f,7,6,1,0,NaN
3,24805,https://www.airbnb.com/rooms/24805,20210413152844,2021-04-15,Gran Via Studio Madrid,"Studio located 50 meters from Gran Via, next t...","The area is next to the Gran Via, so people li...",https://a0.muscache.com/pictures/miso/Hosting-...,346366726,https://www.airbnb.com/users/show/346366726,...,10.0,10.0,10.0,NaN,t,1,1,0,0,0.13
4,26825,https://www.airbnb.com/rooms/26825,20210413152844,2021-04-15,Single Room whith private Bathroom,Nice and cozy roon for one person with a priva...,"Es un barrio muy tranquilo, en una zona de Mad...",https://a0.muscache.com/pictures/149358/218d5b...,114340,https://www.airbnb.com/users/show/114340,...,9.0,9.0,9.0,NaN,f,1,0,1,0,1.12


Comenzaremos borrando una serie de variables que no nos proporcionan información: listing_url, scrape_id, last_scraped, neighborhood_overview, picture_url, host_url, host_name, host_thumbnail_url, host_picture_url, neighbourhood, host_neighbourhood, etc.

In [40]:
df_listings_det_v2 = df_listings_det.drop(columns = ["listing_url","scrape_id","last_scraped",
                                                     "neighborhood_overview","picture_url","host_url",
                                                     "host_name", "host_since", "host_location",
                                                     "host_thumbnail_url", "host_picture_url",
                                                     "neighbourhood", "host_neighbourhood", "minimum_minimum_nights",
                                                     "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights",
                                                     "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", "calendar_updated",
                                                     "calendar_last_scraped", "number_of_reviews_ltm", "number_of_reviews_l30d",
                                                     "first_review", "last_review", "license", "calculated_host_listings_count_entire_homes",
                                                     "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"])

In [41]:
df_listings_det_v2.isna().sum()

id                                    0
name                                  3
description                        1138
host_id                               0
host_about                        10256
host_response_time                 8477
host_response_rate                 8477
host_acceptance_rate               8602
host_is_superhost                   527
host_listings_count                 527
host_total_listings_count           527
host_verifications                    0
host_has_profile_pic                527
host_identity_verified              527
neighbourhood_cleansed                0
neighbourhood_group_cleansed          0
latitude                              0
longitude                             0
property_type                         0
room_type                             0
accommodates                          0
bathrooms                         19618
bathrooms_text                       29
bedrooms                           1495
beds                                400


Existen variables con valores nulos para mas del 30% de las filas, por lo que las borraremos

In [42]:
df_listings_det_v3 = df_listings_det_v2.drop(columns = ["review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness",
                                                        "review_scores_checkin", "review_scores_communication", "review_scores_location",
                                                        "review_scores_value", "host_about", "host_response_time", "host_response_rate",
                                                        "host_acceptance_rate"])

In [43]:
df_listings_det_v3['reviews_per_month'] = df_listings_det_v3['reviews_per_month'].fillna(0)
df_listings_det_v3 = df_listings_det_v3[df_listings_det_v3['bedrooms'].notna()]
df_listings_det_v3 = df_listings_det_v3[df_listings_det_v3['beds'].notna()]
df_listings_det_v3 = df_listings_det_v3[df_listings_det_v3['bathrooms_text'].notna()]
df_listings_det_v3 = df_listings_det_v3[df_listings_det_v3['name'].notna()]
df_listings_det_v3 = df_listings_det_v3[df_listings_det_v3['description'].notna()]
df_listings_det_v3 = df_listings_det_v3[df_listings_det_v3['host_has_profile_pic'].notna()]

In [44]:
df_listings_det_v3.isna().sum()

id                                    0
name                                  0
description                           0
host_id                               0
host_is_superhost                     0
host_listings_count                   0
host_total_listings_count             0
host_verifications                    0
host_has_profile_pic                  0
host_identity_verified                0
neighbourhood_cleansed                0
neighbourhood_group_cleansed          0
latitude                              0
longitude                             0
property_type                         0
room_type                             0
accommodates                          0
bathrooms                         16382
bathrooms_text                        0
bedrooms                              0
beds                                  0
amenities                             0
price                                 0
minimum_nights                        0
maximum_nights                        0


In [45]:
def bathrooms_float(x):
  try:
    y = float(x.split(' ')[0])
  except:
    y = "Nan"
  return y

In [46]:
df_listings_det_v3['bathrooms'] = df_listings_det_v3['bathrooms_text'].apply(lambda x: bathrooms_float(x))

In [47]:
df_listings_det_v3['bathrooms'][df_listings_det_v3['bathrooms'] == "Nan"].count()

25

In [48]:
df_listings_det_vf = df_listings_det_v3[df_listings_det_v3['bathrooms'] != "Nan"]

In [49]:
df_listings_det_vf.isna().sum()

id                                0
name                              0
description                       0
host_id                           0
host_is_superhost                 0
host_listings_count               0
host_total_listings_count         0
host_verifications                0
host_has_profile_pic              0
host_identity_verified            0
neighbourhood_cleansed            0
neighbourhood_group_cleansed      0
latitude                          0
longitude                         0
property_type                     0
room_type                         0
accommodates                      0
bathrooms                         0
bathrooms_text                    0
bedrooms                          0
beds                              0
amenities                         0
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
availability_30                   0
availability_60             